## 1 Android编译原理
### 编译过程示意图
![](image/2021-12-11-11-29-01.png)

1. 打包资源文件，生成R.java文件。
2. 处理aidl文件，生成相应的Java文件。
2. 编译工程源代码，生成相应的class文件。
2. 转换所有的class文件，生成classes.dex文件。
2. 打包生成APK文件。

下面是更详细的编译示意图
![](image/2021-12-12-19-56-23.png)

### 编译过程文件
1. Java文件-----应用程序源文件。Android本身相当一部分都是用java编写而成
2. Class文件------Java编译后的目标文件.不像J2se，java编译成class就可以直接运行，android平台上class文件不能直接在android上运行。android的class文件实际上只是编译过程中的中间目标文件，需要链接成dex文件后才能在dalvik上运行
3. Dex文件-----Android平台上的可执行文件.Android虚拟机Dalvik支持的字节码文件格式Google在新发布的Android平台上使用了自己的Dalvik虚拟机来定义， 这种虚拟机执行的并非Java字节码， 而是dex格式的字节码。在编译Java代码之后，通过Android平台上的工具可以将Java字节码转换成Dex字节码。
4. Apk文件-------Android上的安装文件Apk是Android安装包的扩展名，一个Android安装包包含了与某个Android应用程序相关的所有文件。apk文件将AndroidManifest.xml文件、应用程序代码(.dex文件)、资源文件和其他文件打成一个压缩包。一个工程只能打进一个.apk文件


### 相关工具介绍
* APT（annotation processing tool）是一种处理Java/android注解的工具，它对源代码文件进行检测找出其中的Annotation后，对Annotation进行额外的处理。Annotation处理器在处理Annotation时可以根据源文件中的Annotation生成额外的源文件和其它的文件（文件具体内容由Annotation处理器的编写者决定），APT还会编译生成的源代码文件和原来的源文件，将它们一起生成class文件。代码可能用到了面向切面变成，可能有大量的注解，通过APT处理源代码，讲注解展开，生成新的源代码。
* AspectJ最常用的字节码处理框架有 AspectJ、ASM 等等，它们的相同之处在于输入输出都是 Class 文件。并且，它们都是 在 Java 文件编译成 .class 文件之后，生成 Dalvik 字节码之前执行。AspectJ是一个面向切面的框架，它扩展了Java语言。AspectJ定义了AOP语法，它有一个专门的编译器用来生成遵守Java字节编码规范的Class文件。
* 

### 一些代码防护措施

* 代码混淆
* 核心方法Native
* 动态加载dex，加载dex之前dex文件加密
* 动态加载so
* 所有声明字符串加密
* 使用3方加固平台（治标不治本）
* 插入代码花指令，jui工具编译方法报错

### 反编译的目标

1. 查看动态代码逻辑：将apk里的dex文件转换成Jar包，再通过工具查看编译前的java源文件。
2. 查看静态配置文件：通过工具查看apk里对应的AndroidManifest.xml、resources.arsc、res各布局文件等二进制文件。

### APK文件解析
![](image/2021-12-12-19-41-07.png)

* assets文件夹：原始资源文件夹，对应着Android工程的assets文件夹，一般用于存放原始的图片、txt、css等资源文件。
* lib：存放应用需要的引用第三方SDK的so库。比如一些底层实现的图片处理、音视频处理、数据加密的库等。而该文件夹下有时会多一个层级，这是根据不同CPU 型号而划分的，如 ARM，ARM-v7a，x86等。
* META-INF：保存apk签名信息，保证apk的完整性和安全性。
* res：资源文件夹，其中的资源文件包括了布局(layout)，常量值(values)，颜色值(colors)，尺寸值(dimens)，字符串(strings)，自定义样式(styles)等。
* AndroidManifest.xml文件：全局配置文件，里面包含了版本信息、activity、broadcasts等基本配置。不过这里的是二进制的xml文件，无法直接查看，需要反编译后才能查看。
* classes.dex文件：这是安卓代码的核心部分,，dex是在Dalvik虚拟机上可以执行的文件。这里有classes.dex 4个文件，说明工程的方法数较多，进行了dex拆分。如果apk的方法数超过了65535，会生成多个dex文件，反编译的话需要对这多个dex文件均进行转换Jar包处理。
* resources.arsc文件：记录资源文件和资源id的映射关系。

### 注解说明
运行时注解
通常被定义的注解需要通过反射来获取相关值
编译时注解
在代码构建编译过程的时候,生成java文件然后供需要的类进行调用

两者根本区别在于,前者是程序员预先写好的java文件中,直接调用的,
而后者是程序员写好java代码的生成规则,程序员自己不写java文件,
交给编译器去写java文件,,java文件只有编译器编译完成后才能调用。



## 2 DVM讲解（Android运行时虚拟环境）

> 来自于王还是那个的一篇文章 https://blog.csdn.net/iblade/article/details/78959750

### jvm与dvm对比
![](2021-12-15-14-53-19.png)

1. JVM Java virtual machineJava的虚拟机
   1. 基于栈，指令会更多。
   2. .java文件编译成.class文件，然后打包成jar文件。
   3. JVM加载.jar文件时，回家再所有的.class文件。
2. DVM Dalvik virtual machine Android的虚拟机
   1. 基于寄存器，指令更长，但指令更少。
   2. .java文件编译成.class文件，汇编成.dex文件，然后打包成apk文件
   3. DVM加载apk文件时，只加载一个经过优化后的.dex文件。
   4. 每一个应用都运行在一个独立的DVM实例当中，每一个DVM实例都运行在一个独立的进程空间中。

### dvm静态架构

DVM的源码位于dalvik/目录下，其中dalvik/vm目录下的内容是DVM的具体实现部分，它会被编译成libdvm.so；dalvik/libdex会被编译成libdex.a静态库，作为dex工具使用；dalvik/dexdump是.dex文件的反编译工具；DVM的可执行程序位于dalvik/dalvikvm中，将会被编译成dalvikvm可执行程序。DVM架构如下图所示。

![](2021-12-15-14-58-53.png)

### dvm运行时堆

DVM的运行时堆主要由两个Space以及多个辅助数据结构组成，两个Space分别是Zygote Space（Zygote Heap）和Allocation Space（Active Heap）。Zygote Space用来管理Zygote进程在启动过程中预加载和创建的各种对象，Zygote Space中不会触发GC，所有进程都共享该区域，比如系统资源。Allocation Space是在Zygote进程fork第一个子进程之前创建的，它是一种私有进程，Zygote进程和fock的子进程在Allocation Space上进行对象分配和释放。
除了这两个Space，还包含以下数据结构：

* Card Table： 用于DVM Concurrent GC，当第一次进行垃圾标记后，记录垃圾信息。
* Heap Bitmap： 有两个Heap Bitmap，一个用来记录上次GC存活的对象，另一个用来记录这次GC存活的对象。
* Mark Stack： DVM的运行时堆使用标记-清除（Mark-Sweep）算法进行GC，不了解标记-清除算法的同学查看Java虚拟机（四）垃圾收集算法这篇文章。Mark Stack就是在GC的标记阶段使用的，它用来遍历存活的对象。

### ART虚拟机

ART(Android Runtime)是Android 4.4发布的，用来替换Dalvik虚拟，Android 4.4默认采用的还是DVM，系统会提供一个选项来开启ART。在Android 5.0时，默认采用ART，DVM从此退出历史舞台。

### ART与DVM的区别
1. DVM中的应用每次运行时，字节码都需要通过即时编译器（JIT，just in time）转换为机器码，这会使得应用的运行效率降低。而在ART中，系统在安装应用时会进行一次预编译（AOT，ahead of time）,将字节码预先编译成机器码并存储在本地，这样应用每次运行时就不需要执行编译了，运行效率也大大提升。
2. ART占用空间比Dalvik大（字节码变为机器码之后，可能会增加10%-20%），这就是“时间换空间大法”。
3. 预编译也可以明显改善电池续航，因为应用程序每次运行时不用重复编译了，从而减少了 CPU 的使用频率，降低了能耗。
### ART的运行时堆
与DVM的GC不同的是，ART的GC类型有多种，主要分为Mark-Sweep GC和Compacting GC。ART的运行时堆的空间根据不同的GC类型也有着不同的划分，如果采用的是Mark-Sweep GC，运行时堆主要是由四个Space和多个辅助数据结构组成，四个Space分别是Zygote Space、Allocation Space、Image Space和Large Object Space。Zygote Space、Allocation Space和DVM中的作用是一样的。Image Space用来存放一些预加载类，Large Object Space用来分配一些大对象（默认大小为12k）。其中Zygote Space和Image Space是进程间共享的。
